In [ ]:
!pip install mxnet-cu102
!pip install autogluon

# https://github.com/awslabs/autogluon
!pip install autogluon.tabular[all]

In [21]:
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
df = pd.read_csv('../input/zikavirusdataset/classification_dataset.csv')
df.head()

,density_per_km,latitude,longitude,maxtempC,mintempC,totalSnow_cm,sunHour,moon_illumination,DewPointC,WindGustKmph,cloudcover,humidity,precipMM,pressure,visibility,winddirDegree,year,month,Target
0,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,87,6,16,0,59,0.0,1018,10,189,2016,3,1
1,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,87,6,16,0,59,0.0,1018,10,189,2016,3,0
2,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,87,6,16,0,59,0.0,1018,10,189,2016,3,1
3,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,87,6,16,0,59,0.0,1018,10,189,2016,3,0
4,12625.800781,-34.603684,-58.381559,22,10,0.0,11.5,76,10,27,34,72,0.0,1025,10,99,2016,3,0


In [3]:
df.shape

(155223, 19)

In [4]:
df.duplicated().sum()

79878

In [5]:
# Remove duplicate rows
df = df.drop_duplicates(keep='first', inplace=False)
df.shape

(75345, 19)

In [9]:
train,test = train_test_split(df,test_size=0.40,random_state=0,stratify=df['Target'] ) #train test split in 50:50 ratio

In [10]:
train.shape, test.shape

((45207, 19), (30138, 19))

In [11]:
train.head()

,density_per_km,latitude,longitude,maxtempC,mintempC,totalSnow_cm,sunHour,moon_illumination,DewPointC,WindGustKmph,cloudcover,humidity,precipMM,pressure,visibility,winddirDegree,year,month,Target
122817,4150.221680,19.479196,-70.693057,27,21,0.0,8.7,89,21,5,75,90,4.0,1014,8,68,2016,11,0
66805,111.392563,3.994110,-76.227110,30,17,0.0,12.2,28,13,5,4,60,1.9,1014,10,203,2016,7,1
77788,226.130783,9.039263,-75.790511,33,26,0.0,12.7,68,24,12,24,81,15.6,1008,9,266,2016,8,0
74587,45.342354,5.052433,-73.504646,21,8,0.0,12.6,17,9,11,30,77,0.7,1015,9,116,2016,8,0
95988,69.236809,5.615156,-75.625274,22,16,0.0,8.8,36,17,7,63,94,54.3,1017,6,240,2016,11,1


In [12]:
test.head()

,density_per_km,latitude,longitude,maxtempC,mintempC,totalSnow_cm,sunHour,moon_illumination,DewPointC,WindGustKmph,cloudcover,humidity,precipMM,pressure,visibility,winddirDegree,year,month,Target
85860,49.728394,4.156630,-76.287762,27,16,0.0,12.5,59,14,5,18,68,16.6,1015,9,207,2016,9,1
50844,59.433132,9.144527,-74.226294,31,23,0.0,11.9,100,24,8,57,85,6.2,1010,7,229,2016,5,1
138749,209.804016,15.594288,-90.149499,30,17,0.0,13.0,84,17,7,13,74,1.0,1012,8,140,2016,5,1
107305,147.182251,19.052706,-70.149226,27,18,0.0,6.2,64,19,8,77,82,1.5,1016,7,158,2016,4,0
154932,835.046448,18.335765,-64.896335,28,27,0.0,8.7,96,23,41,42,77,12.9,1017,9,82,2016,12,1


In [14]:
train['Target'].value_counts(), test['Target'].value_counts(),

(0    24440
 1    20767
 Name: Target, dtype: int64,
 0    16294
 1    13844
 Name: Target, dtype: int64)

In [15]:
label = 'Target'
print("Summary of class variable: \n", train[label].describe())

Summary of class variable: 
 count    45207.000000
mean         0.459376
std          0.498352
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: Target, dtype: float64


In [22]:
save_path = 'agModels-predictClass-second'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train)

In [23]:
y_test = test[label]  # values to predict
test_data_nolab = test.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,density_per_km,latitude,longitude,maxtempC,mintempC,totalSnow_cm,sunHour,moon_illumination,DewPointC,WindGustKmph,cloudcover,humidity,precipMM,pressure,visibility,winddirDegree,year,month
85860,49.728394,4.156630,-76.287762,27,16,0.0,12.5,59,14,5,18,68,16.6,1015,9,207,2016,9
50844,59.433132,9.144527,-74.226294,31,23,0.0,11.9,100,24,8,57,85,6.2,1010,7,229,2016,5
138749,209.804016,15.594288,-90.149499,30,17,0.0,13.0,84,17,7,13,74,1.0,1012,8,140,2016,5
107305,147.182251,19.052706,-70.149226,27,18,0.0,6.2,64,19,8,77,82,1.5,1016,7,158,2016,4
154932,835.046448,18.335765,-64.896335,28,27,0.0,8.7,96,23,41,42,77,12.9,1017,9,82,2016,12


In [24]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Predictions:  
 85860     0
50844     0
138749    1
107305    0
154932    1
         ..
76754     0
50884     0
141791    0
22127     0
60368     0
Name: Target, Length: 30138, dtype: int64


In [25]:
perf

{'accuracy': 0.6061782467317008,
 'balanced_accuracy': 0.5841901706319735,
 'mcc': 0.20168986649649448,
 'f1': 0.42257358306981274,
 'precision': 0.647146475935032,
 'recall': 0.31370991043051144}

In [28]:
predictor.leaderboard(test, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,0.606975,0.6136,0.094865,0.018707,0.854956,0.094865,0.018707,0.854956,1,True,4
1,LightGBMLarge,0.606377,0.6144,0.083294,0.017066,1.072196,0.083294,0.017066,1.072196,1,True,13
2,WeightedEnsemble_L2,0.606178,0.6212,1.823909,0.409543,151.926538,0.006929,0.021904,2.384581,2,True,14
3,CatBoost,0.601732,0.6076,0.008947,0.008843,2.127213,0.008947,0.008843,2.127213,1,True,7
4,XGBoost,0.597253,0.6088,0.070364,0.015816,2.345044,0.070364,0.015816,2.345044,1,True,11
5,LightGBMXT,0.595726,0.6040,0.050025,0.014901,2.219124,0.050025,0.014901,2.219124,1,True,3
6,NeuralNetFastAI,0.594797,0.6072,0.526498,0.066544,56.772639,0.526498,0.066544,56.772639,1,True,10
7,NeuralNetMXNet,0.578008,0.5704,0.982987,0.245763,84.150786,0.982987,0.245763,84.150786,1,True,12
8,KNeighborsUnif,0.479859,0.4688,1.416723,0.213832,0.126788,1.416723,0.213832,0.126788,1,True,1
9,RandomForestEntr,0.414958,0.4176,1.880056,0.309853,30.716294,1.880056,0.309853,30.716294,1,True,6


In [26]:
results = predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2     0.6212       0.409543  151.926538                0.021904           2.384581            2       True         14
1         LightGBMLarge     0.6144       0.017066    1.072196                0.017066           1.072196            1       True         13
2              LightGBM     0.6136       0.018707    0.854956                0.018707           0.854956            1       True          4
3               XGBoost     0.6088       0.015816    2.345044                0.015816           2.345044            1       True         11
4              CatBoost     0.6076       0.008843    2.127213                0.008843           2.127213            1       True          7
5       NeuralNetFastAI     0.6072       0.066544   56.772639                0.066544          56.